# 1. Overview
NASA services can be queried from Python in multiple ways.
* Generic Virtual Observatory (VO) queries.
  * Call sequence is consistent, including for non-NASA resources.
  * Use the `pyvo` package: https://pyvo.readthedocs.io/en/latest/
  * Known issues/caveats: https://github.com/NASA-NAVO/aas_workshop_2020_winter/blob/master/KNOWN_ISSUES.md
* Astroquery interfaces
  * Call sequences not quite as consistent, but follow similar patterns.
  * See https://astroquery.readthedocs.io/en/latest/
* Ad hoc archive-specific interfaces

# 2. VO Services

In [ ]:
# Generic VO access routines
import pyvo as vo

# For specifying coordinates and angles
from astropy.coordinates import SkyCoord
from astropy.units import Quantity


# For downloading files
from astropy.utils.data import download_file

# Ignore unimportant warnings
import warnings
warnings.filterwarnings('ignore', '.*Unknown element mirrorURL.*', vo.utils.xml.elements.UnknownElementWarning)

## 2.1 Lookup Services in VO Directory (Registry)

In [ ]:
# Simple example:  Find Simple Cone Search (conesearch) services related to SWIFT.
services = vo.regsearch(servicetype='conesearch',keywords=['swift'])
services_table = services.to_table()
services_table['short_name','res_title', 'res_description']

### 2.1.1 Use different arguments/values to modify the simple example
| Argument | Description | Examples |
| :-----: | :-----------: | :--------: |
| **servicetype** | Type of service | ‘conesearch’, ‘sia’ , ‘ssa’, ‘slap’, ‘tap’ |
| **keyword** | Results will contain the keyword in ivoid, title, or description | 'galex', 'swift' |
| **waveband** | Resulting services have data in the specified waveband(s) | ‘radio’, ‘millimeter’, ‘infrared’, ‘optical’, ‘uv’, ‘euv’, ‘x-ray’ ‘gamma-ray’ |

### 2.1.2 Inspect the results.
See http://docs.astropy.org/en/stable/table/ for more on working with Astropy Tables.

In [ ]:
print('Number of rows and the column names:')
print(services_table.info)

print('\nDefault display of Astropy Table:')
print(services_table)

print('\nFirst 3 short names and descriptions:')
print(services_table[:3]['short_name', 'res_title'])

In [ ]:
swiftuvot_services = services_table[services_table['short_name'] == b'SwiftUVOT'] # Filter the results by column value.
swiftuvot_services['short_name', 'res_description']

## 2.2 Cone search
Example:  Find a cone search service for the USNO-B catalog and search it around M51 with a .1 degree radius.  (More inspection could be done on the service list instead of blindly choosing the first service.)  The *coords* argument can be anything accepted by: https://astroquery.readthedocs.io/en/latest/api/astroquery.utils.parse_coordinates.html#astroquery.utils.parse_coordinates

More information on creating Astropy SkyCoord objects: http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html

In [ ]:
m51_pos = SkyCoord.from_name("m51")

services = vo.regsearch(servicetype='conesearch', keywords='usno-b')
results = services[0].search(pos=m51_pos, radius=0.1)
results.to_table()

## 2.3 Image search
Example:  Find an image search service for GALEX, and search it around coordinates 13:37:00.950,-29:51:55.51 (M83) with a radius of .2 degrees.  Download the first file in the results.
#### Find an image service

In [ ]:
services = vo.regsearch(servicetype='image', keywords=['galex'])
services.to_table()['ivoid', 'short_name', 'res_title']

#### Search one of the services
The first service looks good.  Search it!

For more details on using `SkyCoord` see http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord

In [ ]:
m83_pos = SkyCoord('13h37m00.950s -29d51m55.51s')
results = services[0].search(pos=m83_pos, size=.2)

# We can look at the results.
results.to_table()

#### Download an image
For the first result, print the file format and download the file. If repeatedly executing this code, add `cache=True` to `download_file()` to prevent repeated downloads.

See `download_file()` documentation here: https://docs.astropy.org/en/stable/api/astropy.utils.data.download_file.html#astropy.utils.data.download_file

In [ ]:
print(results[0].format)
file_name = download_file(results[0].getdataurl())  
file_name

## 2.4 Spectral search
Example:  Find a spectral service for x-ray data.  Query it around Delta Ori with a search **diameter** of 10 arc minutes, and download the first data product.  Note that the results table can be inspected for potentially useful columns.

Spectral search is very similar to image search. In this example, note:
* **`diameter` defines the size of the search region**
* `waveband` used in `regsearch()`
* Astropy `Quantity` used to specify radius units other than degrees.

In [ ]:
# Search for a spectrum search service that has x-ray data.
services = vo.regsearch(servicetype='spectrum', waveband='x-ray')

# Assuming there are services and the first one is OK...
results = services[0].search(pos=SkyCoord.from_name("Delta Ori"), 
                             diameter=Quantity(10, unit="arcmin"))

# Assuming there are results, download the first file.
print(f'Title: {results[0].title}, Format: {results[0].format}')
file_name = download_file(results[0].getdataurl())  
file_name

## 2.5 Table search
Example:  Find the HEASARC Table Access Protocol (TAP) service, get some information about the available tables.

In [ ]:
services = vo.regsearch(servicetype='tap', keywords='heasarc')
print(f'{len(services)} service(s) found.')

# We found only one service.  Print some info about the service and its tables.
print(f'Title: {services[0].res_title}')
print(f'{services[0].res_description}')
      
tables = services[0].service.tables  # Queries for details of the service's tables
print(f'{len(tables)} tables:')
for t in tables:
    print(f'{t.name:30s} - {t.description}')  # A more succinct option than t.describe()

#### Column Information
For any table, we can list the column names and descriptions.

In [ ]:
for c in tables['zcat'].columns:
    print(f'{c.name:30s} - {c.description}')

#### Perform a Query
Example:  Perform a cone search on the ZCAT catalog.

# 3. Astroquery 
Many archives have Astroquery modules for data access, including:

* [HEASARC Queries (astroquery.heasarc)](https://astroquery.readthedocs.io/en/latest/heasarc/heasarc.html)
* [HITRAN Queries (astroquery.hitran)](https://astroquery.readthedocs.io/en/latest/hitran/hitran.html)
* [IRSA Image Server program interface (IBE) Queries (astroquery.ibe)](https://astroquery.readthedocs.io/en/latest/ibe/ibe.html)
* [IRSA Queries (astroquery.irsa)](https://astroquery.readthedocs.io/en/latest/irsa/irsa.html)
* [IRSA Dust Extinction Service Queries (astroquery.irsa_dust)](https://astroquery.readthedocs.io/en/latest/irsa/irsa_dust.html)
* [JPL Spectroscopy Queries (astroquery.jplspec)](https://astroquery.readthedocs.io/en/latest/jplspec/jplspec.html)
* [MAST Queries (astroquery.mast)](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
* [NASA ADS Queries (astroquery.nasa_ads)](https://astroquery.readthedocs.io/en/latest/nasa_ads/nasa_ads.html)
* [NED Queries (astroquery.ned)](https://astroquery.readthedocs.io/en/latest/ned/ned.html)

For more, see https://astroquery.readthedocs.io/en/latest/

## 3.1 NED
Example:  Get an Astropy Table containing the objects from paper 2018ApJ...858...62K.  For more on the API, see https://astroquery.readthedocs.io/en/latest/ned/ned.html 